In [2]:
from AttentionModule import Conv2d_Attn

import torch
from torch import nn
from torchvision import models, datasets, transforms
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

import re
import numpy as np
import time
from tqdm import tqdm
import copy

In [3]:
resnet_pretrained = models.resnet50(pretrained=True)
nn.Conv2d = Conv2d_Attn
resnet_attn = models.resnet50()
    
resnet_attn.load_state_dict(resnet_pretrained.state_dict(), strict=False)
# print(resnet_pretrained.state_dist())

In [5]:
# This block turns 'layer1.0.downsample.0.weight' to 'layer1[0].downsample[0].weight'
param_keys = list(resnet_attn.state_dict().keys())
formatted_keys = []
for k in param_keys:
    found = re.findall(r'\.[\d]{1,2}\.', k)
    if len(found):
        for f in found:
            k = k.replace(f, '[{}].'.format(f.strip('.')))
    formatted_keys.append(k)

In [6]:
# This block turn off gradient up for all params except attn_weights
def turn_off_grad_except(lst=[]):
    turning_off_keys = [k for k in formatted_keys for l in lst if l in k]
    for k in formatted_keys:
        obj = eval('resnet_attn.'+k)
        if k in turning_off_keys:
            print(k)
            obj.requires_grad = True
        else:
            obj.requires_grad = False
    

def turn_on_layer4_conv_weight():
    for k in range(3):
        for j in range(1, 4):
            eval('resnet_attn.layer4[{}].conv{}.weight'.format(k, j)).requires_gred = True

In [7]:
resnet_attn.fc = nn.Linear(resnet_attn.fc.in_features, 144)

Start training

In [8]:
batch_size = 32

In [9]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose(
    [transforms.ToTensor(),
     normalize])

trainset = torchvision.datasets.ImageFolder(root='./data/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [10]:
total_imgs = len(trainset.imgs)

In [11]:
resnet_attn = resnet_attn.cuda()

In [12]:
total_attn_params = 0
for k in formatted_keys:
    obj = eval('resnet_attn.'+k)
    if 'attn_weights' in k:
        total_attn_params += np.prod(obj.shape)
print("Total number of attention parameters", total_attn_params)

Total number of attention parameters 26560


We want the attention parameters to diverge from 1, therefore we penalize element-wise square loss as $\lambda (1 \times \text{# params} - (x - 1)^2)$

But this is too big a number,
let's try: 
$- (x - 1)^2$ for now

In [13]:
_lambda = 1e-1 #set default

In [14]:
def get_params_objs(name, net='resnet_attn'):
    res = []
    for k in formatted_keys:
        obj = eval('{}.'.format(net)+k)
        if name in k:
            res.append(obj)
    return res

In [15]:
def compute_attn_loss(n_params=26560):
    attns = get_params_objs('attn_weights')
#     penality = sum([torch.pow(t - 1,2).mean() for t in attns])
    penality = (1000/n_params)*sum([torch.min(torch.pow(t-2, 2), torch.pow(t, 2)).sum() for t in attns])
    return (_lambda)*(penality)

In [16]:
len(get_params_objs('attn_weights'))

53

In [17]:
compute_attn_loss()

tensor(100.0000, device='cuda:0')

In [18]:
print_every = 20

In [19]:
def train_k_epoch(k, add_attn=True, score_epoch=False):
    cls_criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet_attn.parameters()))
    
    start = time.time()
    
    best_accu = 0
    best_model_wts = {}
    
    for epoch in range(k):
        running_loss = 0.0
        running_attn_loss = 0.0

        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()

            optimizer.zero_grad()
            outputs = resnet_attn(inputs)
            clf_loss = cls_criterion(outputs, labels)
                        
            attn_loss = compute_attn_loss()
            
            if add_attn:
                loss = clf_loss + attn_loss
            else:
                loss = clf_loss

            loss.backward()
            optimizer.step()

            running_loss += clf_loss.data[0]
            running_attn_loss += attn_loss.data[0]

            if i % print_every == 0:
                print('[%5d] iter, [%2f] epoch, classifer loss: %.3f, attn_loss: %.5f' %
                      (i + 1, i*batch_size/total_imgs, running_loss/print_every, running_attn_loss/print_every))
                running_loss = 0.0
                running_attn_loss = 0.0
        if score_epoch:
            dic = score(batch_size=32)
            e_time = time.time() - start
            dic['time_elapsed'] = '{:.0f}m {:.0f}s'.format(e_time // 60, e_time%60)
            print(dic)
            
            if dic['val_accu_top1'] > best_accu:
                best_accu = dic['val_accu_top1']
                best_model_wts = copy.deepcopy(resnet_attn.state_dict())

In [20]:

def score(net=resnet_attn, batch_size=batch_size):
    trainset = torchvision.datasets.ImageFolder(root='./data/train', transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              shuffle=True, num_workers=2)
    
    valset = torchvision.datasets.ImageFolder(root='./data/val', transform=transform)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                              shuffle=True, num_workers=2)
    
    train_correct = 0
    val_correct_top1 = 0
    val_correct_top3 = 0
    val_correct_top5 = 0
    
    for inp, label in tqdm(iter(valloader)):
        _, idx = net(Variable(inp).cuda()).topk(1)
        val_correct_top1 += int(sum([label[i] in idx.cpu().data[i] for i in range(len(idx))]))
        
        _, idx = net(Variable(inp).cuda()).topk(3)
        val_correct_top3 += int(sum([label[i] in idx.cpu().data[i] for i in range(len(idx))]))
        
        _, idx = net(Variable(inp).cuda()).topk(5)
        val_correct_top5 += int(sum([label[i] in idx.cpu().data[i] for i in range(len(idx))]))
        
    for inp, label in tqdm(iter(trainloader)):
        _, idx = net(Variable(inp).cuda()).topk(1)
        train_correct += int(sum([label[i] in idx.cpu().data[i] for i in range(len(idx))]))
    
    return {
        'train_accu': train_correct/len(trainset),
        'val_accu_top1': val_correct_top1/len(valset),
        'val_accu_top3': val_correct_top3/len(valset),
        'val_accu_top5': val_correct_top5/len(valset)
    }

Train a fresh fc layer. 
`turn_off_grad_except([])` turns off grads for all weights but the fc layer

In [21]:
turn_off_grad_except(['fc', 'attn_weights'])
turn_on_layer4_conv_weight()
resnet_attn.eval()
resnet_attn.layer4.train()
train_k_epoch(50,score_epoch=True, add_attn=True)

conv1.attn_weights
layer1[0].conv1.attn_weights
layer1[0].conv2.attn_weights
layer1[0].conv3.attn_weights
layer1[0].downsample[0].attn_weights
layer1[1].conv1.attn_weights
layer1[1].conv2.attn_weights
layer1[1].conv3.attn_weights
layer1[2].conv1.attn_weights
layer1[2].conv2.attn_weights
layer1[2].conv3.attn_weights
layer2[0].conv1.attn_weights
layer2[0].conv2.attn_weights
layer2[0].conv3.attn_weights
layer2[0].downsample[0].attn_weights
layer2[1].conv1.attn_weights
layer2[1].conv2.attn_weights
layer2[1].conv3.attn_weights
layer2[2].conv1.attn_weights
layer2[2].conv2.attn_weights
layer2[2].conv3.attn_weights
layer2[3].conv1.attn_weights
layer2[3].conv2.attn_weights
layer2[3].conv3.attn_weights
layer3[0].conv1.attn_weights
layer3[0].conv2.attn_weights
layer3[0].conv3.attn_weights
layer3[0].downsample[0].attn_weights
layer3[1].conv1.attn_weights
layer3[1].conv2.attn_weights
layer3[1].conv3.attn_weights
layer3[2].conv1.attn_weights
layer3[2].conv2.attn_weights
layer3[2].conv3.attn_weights


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:33: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[    1] iter, [0.000000] epoch, classifer loss: 0.256, attn_loss: 5.00000
[   21] iter, [0.078422] epoch, classifer loss: 4.690, attn_loss: 97.94265
[   41] iter, [0.156844] epoch, classifer loss: 4.291, attn_loss: 94.12140
[   61] iter, [0.235265] epoch, classifer loss: 3.915, attn_loss: 90.41732
[   81] iter, [0.313687] epoch, classifer loss: 3.793, attn_loss: 86.83248
[  101] iter, [0.392109] epoch, classifer loss: 3.597, attn_loss: 83.37243
[  121] iter, [0.470531] epoch, classifer loss: 3.679, attn_loss: 80.03396
[  141] iter, [0.548952] epoch, classifer loss: 3.450, attn_loss: 76.81706
[  161] iter, [0.627374] epoch, classifer loss: 3.347, attn_loss: 73.70966
[  181] iter, [0.705796] epoch, classifer loss: 3.520, attn_loss: 70.71809
[  201] iter, [0.784218] epoch, classifer loss: 3.468, attn_loss: 67.83167
[  221] iter, [0.862639] epoch, classifer loss: 3.379, attn_loss: 65.04615
[  241] iter, [0.941061] epoch, classifer loss: 3.514, attn_loss: 62.35752


100%|██████████| 256/256 [01:07<00:00,  3.79it/s]

{'val_accu_top1': 0.2331812998859749, 'val_accu_top3': 0.3973774230330673, 'time_elapsed': '5m 12s', 'val_accu_top5': 0.47206385404789053, 'train_accu': 0.2609974267859331}


...

100%|██████████| 256/256 [01:07<00:00,  3.82it/s]

{'val_accu_top1': 0.16875712656784492, 'val_accu_top3': 0.3312428734321551, 'time_elapsed': '135m 52s', 'val_accu_top5': 0.41961231470923605, 'train_accu': 0.44161254748192624}


[    1] iter, [0.000000] epoch, classifer loss: 0.112, attn_loss: 0.05952
[   21] iter, [0.078422] epoch, classifer loss: 2.239, attn_loss: 1.18955
[   41] iter, [0.156844] epoch, classifer loss: 2.037, attn_loss: 1.18814
[   61] iter, [0.235265] epoch, classifer loss: 2.152, attn_loss: 1.18886
[   81] iter, [0.313687] epoch, classifer loss: 2.186, attn_loss: 1.18869
[  101] iter, [0.392109] epoch, classifer loss: 2.164, attn_loss: 1.18980
[  121] iter, [0.470531] epoch, classifer loss: 2.151, attn_loss: 1.19090
[  141] iter, [0.548952] epoch, classifer loss: 2.240, attn_loss: 1.19059
[  161] iter, [0.627374] epoch, classifer loss: 2.231, attn_loss: 1.18905


Process Process-158:
Process Process-157:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kw

KeyboardInterrupt: 

100%|██████████| 55/55 [00:14<00:00,  3.79it/s]


{'train_accu': 0.0, 'val_accu': 0.5872291904218928}

In [ ]:
compute_attn_loss()

In [ ]:
_lambda = 0.01

In [ ]:
turn_off_grad_except(['attn_weights','fc', 'layer4'])
resnet_attn.eval()
train_k_epoch(5,score_epoch=True)

In [28]:
3

3